In [4]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [5]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio

In [6]:
iOption0 = 1
part = 0
N_files = 4
N_loc_files = 2

In [7]:
N_centr = 5
N_pt = 1
#colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,1.5,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
colors = [3,1,2,4,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="../sim/output/invmass_embed/"
file_names=["bbbar_v1_10M.root","ccbar_v4_12M.root","phi_v9_10M.root","jpsi_v6_10M.root","photon_v2_50M.root"]
part_names = ["b#bar{b}","c#bar{c}","#phi","J/#psi","#gamma#to ee"]
hist_mass_DCA_names = ["inv_mass_dca_fg5","inv_mass_dca_fg7"]
hist_DCA2_names = ["DCA2_hist_0","DCA2_hist_4"]
hist_DCA12_names=["DCA12_hist_0","DCA12_hist_4"]
hists_all = [hist_mass_DCA_names,hist_DCA2_names,hist_DCA12_names]
N_type = len(hists_all)

In [8]:
hists_read = []

for iFile in range(N_files):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [9]:
legends0 = []
def single_DCA_draw(clock, x = 0.5, y = 1.5, iter= 0):
    type_loc =1
    N_dca_files = 4
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c1_{clock}",f"c1_{clock}",1400,450*1)
    c0.Divide(2,1)
    for itype in range(N_type_loc):
        if iter == 1 : legends0.append(root.TLegend(0.65,0.5,0.91,0.88))
        if iter == 1 : Format_Legend(legends0[-1],0.03)
        for iFile in range(0,N_dca_files):
            c0.cd(itype+1)
            ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(x)
            ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(y)
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",1,4,ibin0,ibin1)
            clr = colors[iFile]
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            project.GetXaxis().SetRange(project.FindBin(-500),project.FindBin(500))
            #root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            if iter == 1 : legends0[-1].AddEntry(project,part_names[iFile],"l")
        legends0[-1].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/single_dca.png")
    return c0

In [ ]:
legends, titles = [], []
def pair_DCA_draw_note(clock, x = 0.5, y = 1.5, iter = 0):
    ipt = 2
    type_loc =0
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",800,450*1)
    c0.Divide(1,1)
    N_dca_files = 4
    for itype in range(N_type_loc):
        max = 0

        if iter == 1 : 
            legends.append(root.TLegend(0.75,0.6,0.95,0.95))
            Format_Legend(legends[-1],0.07)
            titles.append(Draw_Legend_Title(0.3,0.78,0.75,0.88,"1.0 < m_{ee} [GeV] < 2.9",0.075))
            titles[-1].Draw()
            titles.append(Draw_Legend_Title(0.3,0.88,0.75,0.98,"Au+Au mb sim",0.075))
        for iFile in range(0,N_dca_files):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            clr = colors[iFile]
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
            root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            if iter == 1 : legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[itype].Draw()
        titles[itype].Draw()


    c0.Draw()

    return c0

In [ ]:
legends, titles = [], []
def pair_DCA_draw(clock, x = 0.5, y = 1.5, iter = 0):
    ipt = 2
    type_loc =0
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",1400,450*1)
    c0.Divide(2,1)
    N_dca_files = 4
    for itype in range(N_type_loc):
        max = 0

        if iter == 1 : 
            legends.append(root.TLegend(0.75,0.6,0.95,0.95))
            Format_Legend(legends[-1],0.03)
            titles.append(Draw_Legend_Title(0.3,0.85,0.75,0.95,"Au+Au mb sim",0.075))
        for iFile in range(0,N_dca_files):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            clr = colors[iFile]
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
            root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            if iter == 1 : legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[itype].Draw()
        titles[itype].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/dca2.png")

    return c0

In [ ]:
if False:
    c = pair_DCA_draw_kek(1,1.0,2.9,1)
    c.Draw()
    c.SaveAs(f"output/dca2.pdf")

In [12]:
clock = 0

In [13]:

clock += 1

slider2 = widgets.FloatRangeSlider(
    value=[1, 3.5],
    min=0.1,
    max=10.0,
    step=0.1,
    description='pT [GeV]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='50%'),
)

async def test2(slider2, output):
    i = 0
    while slider2.value[1] < 9:
        i = i+1
        await asyncio.sleep(1)
        output.update('test2 ' + str(slider2.value) + ' - ' + str(i))
        c1= single_DCA_draw(clock, slider2.value[0],slider2.value[1], i)
        if i == 1: c1.Draw()
        else : c1.Update()
    return

output = display("tbd", display_id=True)
display(slider2)


task = asyncio.create_task(test2(slider2, output))

'tbd'

FloatRangeSlider(value=(1.0, 3.5), continuous_update=False, description='pT [GeV]:', layout=Layout(width='50%'…

In [14]:
if True: was_cancelled = task.cancel()

In [15]:

clock += 1

slider2 = widgets.FloatRangeSlider(
    value=[1, 3.5],
    min=0.1,
    max=10.0,
    step=0.1,
    description='Mass [GeV]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='50%'),
)

async def test2(slider2, output):
    i = 0
    while slider2.value[1]< 9 and slider2.value[0] < 9:
        i = i+1
        await asyncio.sleep(1)
        output.update('test2 ' + str(slider2.value) + ' - ' + str(i))
        c1= pair_DCA_draw(clock, slider2.value[0],slider2.value[1], i)
        if i == 1: c1.Draw()
        else : c1.Update()
    return

output = display("tbd", display_id=True)
display(slider2)


task2 = asyncio.create_task(test2(slider2, output))

'tbd'

FloatRangeSlider(value=(1.0, 3.5), continuous_update=False, description='Mass [GeV]:', layout=Layout(width='50…

In [16]:
if True: was_cancelled = task2.cancel()

import ipywidgets as widgets
import asyncio

slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

async def test(slider, output):
    i = 0
    while slider.value != 10:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value 
        c1= single_DCA_draw(slider.value)[0]
        if i == 1: c1.Draw()
        else : c1.Update()

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))